# CNN LSTM

**Reference:** https://github.com/vinayakumarr/Network-Intrusion-Detection

**Dataset:** http://kdd.ics.uci.edu/databases/kddcup99/kddcup99.html

In [1]:
import numpy as np
np.random.seed(1337)  # for reproducibility

import pandas as pd

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, Dense, Dropout, MaxPooling1D, LSTM

from sklearn.preprocessing import Normalizer

In [13]:
from utils.preprocess import train_and_evaluate_model

def build_model(activation):
    model = Sequential()
    model.add(Conv1D(2, 5, padding="same", activation=activation, input_shape=X_train.shape[1:]))
    model.add(MaxPooling1D(1))
    model.add(LSTM(1))
    model.add(Dropout(0.1))
    model.add(Dense(4, activation="softmax"))
    model.compile(loss="sparse_categorical_crossentropy", optimizer="adam",metrics=['accuracy'])
    return model

In [14]:
DATA = '../data/'
MODELS = '../models/'
TEST = 0.3

In [15]:
traindata = pd.read_csv(DATA+'kddcup.data.csv')
traindata

,0,1,2,3,4,5,6,7,8,9,...,32,33,34,35,36,37,38,39,40,41
0,0,tcp,http,SF,215,45076,0,0,0,0,...,0,0.0,0.0,0.00,0.00,0.0,0.00,0.0,0.0,normal.
1,0,tcp,http,SF,162,4528,0,0,0,0,...,1,1.0,0.0,1.00,0.00,0.0,0.00,0.0,0.0,normal.
2,0,tcp,http,SF,236,1228,0,0,0,0,...,2,1.0,0.0,0.50,0.00,0.0,0.00,0.0,0.0,normal.
3,0,tcp,http,SF,233,2032,0,0,0,0,...,3,1.0,0.0,0.33,0.00,0.0,0.00,0.0,0.0,normal.
4,0,tcp,http,SF,239,486,0,0,0,0,...,4,1.0,0.0,0.25,0.00,0.0,0.00,0.0,0.0,normal.
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4898426,0,tcp,http,SF,212,2288,0,0,0,0,...,255,1.0,0.0,0.33,0.05,0.0,0.01,0.0,0.0,normal.
4898427,0,tcp,http,SF,219,236,0,0,0,0,...,255,1.0,0.0,0.25,0.05,0.0,0.01,0.0,0.0,normal.
4898428,0,tcp,http,SF,218,3610,0,0,0,0,...,255,1.0,0.0,0.20,0.05,0.0,0.01,0.0,0.0,normal.
4898429,0,tcp,http,SF,219,1234,0,0,0,0,...,255,1.0,0.0,0.17,0.05,0.0,0.01,0.0,0.0,normal.


In [16]:
from utils import preprocess
traindata,testdata=preprocess.preprocess(traindata)

In [17]:
class_to_idx={d:i for i,d in enumerate(traindata['Y'].unique())}
traindata['Y']=traindata['Y'].replace(class_to_idx)
testdata['Y']=testdata['Y'].replace(class_to_idx)

In [18]:
X = traindata.drop(columns=['Y'])
Y = traindata['Y']
C = testdata['Y']
T = testdata.drop(columns=['Y'])

In [19]:
scaler = Normalizer().fit(X)
X_train = scaler.transform(X)
X_test = scaler.transform(T)

In [20]:
y_train=Y
y_test= C

In [21]:
# reshape input to be [samples, time steps, features]
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))

In [22]:
# define optimizer and objective, compile model
relu_model=build_model("relu")
tanh_model=build_model("tanh")

In [ ]:
tra

In [23]:
# train
history,report=train_and_evaluate_model(relu_model,X_train,y_train,X_test,y_test)
print(report)

Epoch 1/4
 585/2153 [=======>......................] - ETA: 1:14 - loss: 1.1653 - accuracy: 0.5627

KeyboardInterrupt: 

In [ ]:
tanh_hist,tanh_report=train_and_evaluate_model(tanh_model)
print(tanh_report)

Epoch 1/4


In [ ]:
relu_model.save("../models/relu_cnn_lstm.hdf5")
tanh_model.save("../models/tanh_cnn_lstm.hdf5")